In [1]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

In [174]:
frames = [100, 150, 200, 250, 300] # 읽고싶은 frame (0부터 시작)

원본 영상을 input으로 사용 + 원하는 프레임 정보 혹은
뽑아둔 프레임의 이미지가 모인 디렉토리를 input으로 사용.

In [ ]:
fn_video = "/home/hoseung/Dropbox/DeepInsight/2022/NIA/1Cycle/20220620_213859_.mp4"

In [175]:
# For webcam input:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(fn_video)
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
    
    for iframe in frames: 
        cap.set(cv2.CAP_PROP_POS_FRAMES, iframe)
        success, image = cap.read()
        if not success:
            print("Error? Skipping..")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_org = image.copy()
        results = face_mesh.process(image)

        # Draw the face mesh annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_tesselation_style())
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_contours_style())
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_IRISES,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_iris_connections_style())
        # Flip the image horizontally for a selfie-view display.
        cv2.imwrite(f"frame_{iframe}.png", cv2.flip(image, 1))
        
cap.release()


In [176]:
import numpy as np
def get_xy_points(face_landmarks, iris):
    landmarks = face_landmarks.landmark
    inds = np.unique(list(iris))
    #inds = np.arange(473, 478)
    points = [landmarks[p] for p in inds]
    arr = np.zeros((len(points),2)) 

    nx, ny, _ = image.shape
    for i, pp in enumerate(points):
        arr[i,0] = pp.x * ny
        arr[i,1] = pp.y * nx
    return arr

def get_radius_angle(face_landmarks, detection):
    point = get_xy_points(face_landmarks, detection)
    center = point[0,:]
    x = point[:,0]
    y = point[:,1]
    ra = np.sqrt((x[1] - x[3])**2 + (y[1]-y[3])**2)/2
    rb = np.sqrt((x[2] - x[4])**2 + (y[2]-y[4])**2)/2
    angle = np.arctan((y[3] - y[1]) / (x[1] - x[3])) # note that y axis is flipped
    angle = np.rad2deg(angle)
    return (center, ra, rb, angle)

랜드마크 중 눈과 iris의 index는 아래 문서에서 확인할 수도 있음. 

https://github.com/tensorflow/tfjs-models/blob/838611c02f51159afdd77469ce67f0e26b7bbb23/face-landmarks-detection/src/mediapipe-facemesh/keypoints.ts

참고로, 문서에서의 "leftEyeIris"와 mp_face_mesh.FACEMESH_LEFT_EYE는 서로 다른 눈임. 
하나는 피사체 기준, 하나는 보는 사람 기준인 듯 

### 최종적으로 사용하실 값: center, ra, rb, angle

In [173]:
print("피사체의 왼눈: center, ra, rb, angle\n", 
      get_radius_angle(face_landmarks, [473, 474, 475, 476, 477]))
print("피사체의 오른눈: center, ra, rb, angle\n",
      get_radius_angle(face_landmarks, [468, 469, 470, 471, 472]))

피사체의 왼눈: center, ra, rb, angle
 (array([1487.15858459,  464.30525064]), 37.52503595163705, 28.34770778861998, -3.5628922659378297)
피사체의 오른눈: center, ra, rb, angle
 (array([1138.09593201,  438.00797224]), 34.33496973960113, 27.700583469755063, 2.71608687764671)


테스트용 그림 

In [140]:
from matplotlib.patches import Ellipse

plt.close("all")
fig, ax = plt.subplots(subplot_kw={'aspect': 'equal'})
fig.set_size_inches(16,9)

# 피사체의 왼쪽 눈
center, ra, rb, angle = get_radius_angle(face_landmarks, ind_rightEyeIris)
ellipse1 = Ellipse(center, 2*ra,2*rb, angle=angle, alpha=0.5,
                  facecolor='none', edgecolor="red", lw=3)
ax.add_artist(ellipse1)

# 공막
p_left_eyes = get_xy_points(face_landmarks, mp_face_mesh.FACEMESH_LEFT_EYE)
plt.scatter(p_left_eyes[:,0], p_left_eyes[:,1], s=10)


#피사체의 오른쪽 눈
center, ra, rb, angle = get_radius_angle(face_landmarks, ind_leftEyeIris)
ellipse2 = Ellipse(center, 2*ra,2*rb, angle=angle, alpha=0.5,
                  facecolor='none', edgecolor="red", lw=3)
ax.add_artist(ellipse2)

p_right_eyes = get_xy_points(face_landmarks, mp_face_mesh.FACEMESH_RIGHT_EYE)
plt.scatter(p_right_eyes[:,0], p_right_eyes[:,1], s=10)


ax.imshow(image_org)


plt.show()